In [1]:
#Import Packages
from selenium import webdriver
import time
import pandas as pd
import os
#Import Packages
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import numpy as np

import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
warnings.simplefilter(action='ignore',category=DeprecationWarning)

In [2]:
pages = int(input('Enter number of pages to scrape: '))
page = 0
df = pd.DataFrame(columns=['title','url','company','location','experience'])


for i in range(1,pages+1):
    if i == 1:
        # insert link to page 1 here
        url = "https://www.naukri.com/data-scientist-jobs-?k=data%20scientist&nignbevent_src=jobsearchDeskGNB"
    else:
        # insert link to 2nd page here, replace number 2 with str(i)
        url = "https://www.naukri.com/data-scientist-jobs-in-india"+str(i)+"?k=data%20scientist&nignbevent_src=jobsearchDeskGNB"
    driver = webdriver.Chrome(r"C:\\Users\\prave\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")
    driver.implicitly_wait(10)
    driver.get(url)
    
    time.sleep(3)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    #driver.close()
    
    results = soup.find(class_="list")
    job_elems = results.find_all('article',class_='jobTuple')
    
    for job_elem in job_elems:

        # Job URL
        url = job_elem.find('a',class_='title ellipsis').get('href')

        # Job Title
        title = job_elem.find('a',class_='title ellipsis')

       
        # Company name
        company = job_elem.find('a',class_='subTitle ellipsis fleft')

        # Experience required
        try:
            Exp = job_elem.find('li', class_='fleft br2 placeHolderLi experience')
            Exp_span = Exp.find('span', class_='ellipsis fleft expwdth')
            if Exp_span is None:
                Experience = "Nan"
            else:
                Experience = Exp_span.text
        except AttributeError:
            Experience = "Nan"


        # Location
        Loc = job_elem.find('li',class_='fleft br2 placeHolderLi location')
        Loc_exp = Loc.find('span',class_='ellipsis fleft locWdth')
        if Loc_exp is None:
            continue
        else:
            Location = Loc_exp.text

        try:
            span_tag = job_elem.find('span', {'class': 'styles_jhc__stat__PgY67'})
            if span_tag is None:
                posted_time='Nan'
            else:
                posted_time = span_tag.find('span').text
        except AttributeError:
            posted_time='None'
            
        job_description_section = job_elem.find('section', class_='styles_job-desc-container__txpYf')

        # Extract the job description text
        job_description = job_description_section.get_text(strip=True) if job_description_section else 'nan'

        
        # Adding data to dataframe
        df = df.append({'title':title.text,'url':url,'company':company.text,'experience':Experience,'location':Location}, 
                 ignore_index=True)
    
    page += 1
    print(f'Pages done {page}/{pages}')
    
    driver.close()
    
df.duplicated().sum()

df2 = df.drop_duplicates()

# df2.to_excel('naukri_scraped_data_day_posted.xlsx', index=None)  

Enter number of pages to scrape: 20
Pages done 1/20
Pages done 2/20
Pages done 3/20
Pages done 4/20
Pages done 5/20
Pages done 6/20
Pages done 7/20
Pages done 8/20
Pages done 9/20
Pages done 10/20
Pages done 11/20
Pages done 12/20
Pages done 13/20
Pages done 14/20
Pages done 15/20
Pages done 16/20
Pages done 17/20
Pages done 18/20
Pages done 19/20
Pages done 20/20


In [3]:
df

,title,url,company,location,experience
0,Data Scientist - Manager - P&T Labs,https://www.naukri.com/job-listings-data-scien...,PwC,"Mumbai, Hyderabad/Secunderabad, Bangalore/Beng...",8-12 Yrs
1,GN - Strategy - MC - T&O - NLP Data Science - ...,https://www.naukri.com/job-listings-gn-strateg...,Accenture,"Mumbai, Hyderabad/Secunderabad, Pune, Gurgaon/...",1-3 Yrs
2,GN - Strategy - MC - T&O - NLP Data Science - ...,https://www.naukri.com/job-listings-gn-strateg...,Accenture,"Mumbai, Hyderabad/Secunderabad, Pune, Gurgaon/...",1-4 Yrs
3,Analystics & Modeling Specialist,https://www.naukri.com/job-listings-analystics...,Accenture,"Kolkata, Mumbai, Hyderabad/Secunderabad, Pune,...",6-8 Yrs
4,Data Scientist-Immediate Joiner,https://www.naukri.com/job-listings-data-scien...,Maverick Infotech,"Hybrid - Hyderabad/Secunderabad, Pune, Bangalo...",6-11 Yrs
...,...,...,...,...,...
395,Opportunity For Data Scientists at Deloitte,https://www.naukri.com/job-listings-opportunit...,Deloitte,"Pune, Maharashtra, Mumbai (All Areas)",7-10 Yrs
396,Senior Data Scientist,https://www.naukri.com/job-listings-senior-dat...,Fractal Analytics,"Mumbai, Bangalore/Bengaluru",5-10 Yrs
397,SME Data Scientist,https://www.naukri.com/job-listings-sme-data-s...,Tata Power,"Mumbai, Andheri",6-10 Yrs
398,Data Science+ DL+NLP+ LLM / Langchain Professi...,https://www.naukri.com/job-listings-data-scien...,Wipro,Bangalore/Bengaluru,6-10 Yrs
